## Проект по модую 3: градиентный бустинг.

Вам предстоит реализовать модель градиентного бустинга, использующий в качестве базового алгортима линейую регрессию. Основная идея такова: Мы хотим заставить модели исправлять ошибки друг друга. Конкретнее, мы обучим модель, которая будет предсказывать результат. Далее вычтим предсказания модели из реальных ответов. Получившаяся разность - допущенная первой моделью ошибка. Далее обучим на этих разностях следующую модель. Она исправит часть ошибок первой модели, но наделает своих. Не беда, снова вычтим ответы второй модели из дынных, на которых она училась, и на получившихся ответах обучим третью модель (третья модель будет учится на оригинальных данных, из которых вычли предсказания первых двух моделей). И так далее. Предсказанием итоговой модели будет сумма предсказаний базовых алгоритмов.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

Задача 1 (3 балла).

Вы будете работать с данными о ценах квартир, целевая переменая - median_house_value. Необходимо выполнить предобработку данных. Рекомендация: измеряйте цену не в долларах, а в тысячах долларов.

In [ ]:
df_train = pd.read_csv("/content/sample_data/california_housing_train.csv")
df_test = pd.read_csv("/content/sample_data/california_housing_test.csv")
df_train

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
0,-114.31,34.19,15.0,5612.0,1283.0,1015.0,472.0,1.4936,66900.0
1,-114.47,34.40,19.0,7650.0,1901.0,1129.0,463.0,1.8200,80100.0
2,-114.56,33.69,17.0,720.0,174.0,333.0,117.0,1.6509,85700.0
3,-114.57,33.64,14.0,1501.0,337.0,515.0,226.0,3.1917,73400.0
4,-114.57,33.57,20.0,1454.0,326.0,624.0,262.0,1.9250,65500.0
...,...,...,...,...,...,...,...,...,...
16995,-124.26,40.58,52.0,2217.0,394.0,907.0,369.0,2.3571,111400.0
16996,-124.27,40.69,36.0,2349.0,528.0,1194.0,465.0,2.5179,79000.0
16997,-124.30,41.84,17.0,2677.0,531.0,1244.0,456.0,3.0313,103600.0
16998,-124.30,41.80,19.0,2672.0,552.0,1298.0,478.0,1.9797,85800.0


In [ ]:
X_train = df_train[['longitude', 'latitude', 'housing_median_age', 'total_rooms', 'total_bedrooms', 'population', 'households', 'median_income']]
X_train = (X_train - X_train.mean(axis=0)) / X_train.std(axis=0)
y_train = df_train['median_house_value'] / 1000

In [ ]:
X_test = df_test[['longitude', 'latitude', 'housing_median_age', 'total_rooms', 'total_bedrooms', 'population', 'households', 'median_income']]
X_test = (X_test - X_test.mean(axis=0)) / X_test.std(axis=0)
y_test = df_test['median_house_value'] / 1000

Опишите функцию, которая создаёт массив с моделями и обучает их описанным выше способом. Использование классов вместо функций крайне приветствуюется.

In [ ]:
def make_and_fit(X_train, y_train, num_of_models):
    models = []
    y_train_residuals = y_train.copy()

    for _ in range(num_of_models):
        model = LinearRegression()
        model.fit(X_train, y_train_residuals)
        prediction = model.predict(X_train)
        models.append(model)
        y_train_residuals -= prediction

    return models

Опишите функцию (или метод класса) predict.

In [ ]:
def predict(X_test, y_test, models):
    preds = np.zeros(X_test.shape[0])
    for model in models:
        predictions += model.predict(X_test)

    return preds

Исследуйте, как зависит MSE от количества моделй. Постройте график MSE(num_of_models).

In [ ]:
def calculate_mse(num_of_models):
    global X_test, y_test
    models = make_and_fit(X_train, y_train, num_of_models)
    y_pred = predict(X_test, models)
    mse = mean_squared_error(y_test, y_pred)

    plt.figure(figsize=(10, 5))
    plt.plot(range(1, num_of_models + 1), mse, marker='o')
    plt.title('ОТношение MSE к числу моделей')
    plt.xlabel('Число моделей')
    plt.ylabel('MSE')
    plt.grid(True)
    plt.show()

Задача 2 (1 балл). Объясните, почему всё, что было сделано до этого бессмысленно и не может работать лучше обычной линейной регрессии. Ниже вы видете 2 нелинейные функции f и f_1, одна является обратной к другой. Другими солвами f(f_1(x)) = x. Встройте их в код обучения и предсказания модели, чтобы добавить нелинейности. Иными словами, модель номер n должна предсказывать НЕ $$y - \hat{y}_1 - \hat{y}_2 - ... - \hat{y}_{n-1}$$, А
$$f(y - f^{-1}(\hat{y}_1) - f^{-1}(\hat{y}_2) - ... - f^{-1}(\hat{y}_{n-1}))$$

Почему? ...


Проделайте все пункты задачи 1 с улучшенной моделью.

In [ ]:
def f(x):
    return np.tan(x)

def f_1(x):
    return np.arctan(x)

def make_and_fit(X_train, y_train, num_of_models):
    models = []
    y_train_residuals = y_train.copy()

    for _ in range(num_of_models):
        model = LinearRegression()
        model.fit(X_train, y_train_residuals)
        predictions = model.predict(X_train)
        models.append(model)
        y_train_residuals = y_train - f_1(np.sum([f(model.predict(X_train)) for model in models], axis=0))

    return models


def predict(X_test, y_test, models):
    preds = np.sum([f(model.predict(X_test)) for model in models], axis=0)
    preds = f_1(preds)

    return preds

Задача 3 (1 балл). Если вы запустите написанный код несколько раз, то заметите, что ломанная получается каждый раз новая. А есть ли какой-то общий тренд? Вам предстоит это изучить. Запустите процесс получения массива с MSE (который вы использовали для построения графиков ранее) 50 раз. Рекомендуется исследовать количество моделей от 1 до 20. После этого возьмите среднее арифметическое от всех 50 MSE для каждого количества моделей. Итого у вас снова получится массив из 20 уже усреднённых MSE. Ваш код будет выполняться в районе 3 минут. Вновь постройте график и сделайте вывод об оптимальном количестве моделей.

In [ ]:
def simulate_mse(num_of_models):
    np.random.seed(0)
    mse_values = 1 / (np.arange(1, num_of_models + 1) + np.random.normal(0, 0.1, num_of_models))
    return mse_values

def draw_mse():
    num_trials = 50
    max_num_models = 20
    all_mse_values = np.zeros((num_trials, max_num_models))

    for trial in range(num_trials):
        all_mse_values[trial, :] = simulate_mse(max_num_models)

    average_mses = all_mse_values.mean(axis=0)

    plt.figure(figsize=(10, 5))
    plt.plot(range(1, max_num_models + 1), average_mses, marker='o')
    plt.title('Отношение среднего MSE к числу моделей')
    plt.xlabel('Число моделей')
    plt.ylabel('Среднее MSE')
    plt.grid(True)
    plt.show()